In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K
from sklearn.datasets import fetch_california_housing
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
california_housing = fetch_california_housing(as_frame=True)

2023-03-31 13:01:16.703669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 13:01:16.871932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-31 13:01:16.871953: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-31 13:01:16.902346: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-31 13:01:18.067851: W tensorflow/stream_executor/platform/de

In [31]:
def build_sequential(lr, input_dims, output_dims, layerSizes, activation, loss):
    model = keras.Sequential()
    model.add (keras.layers.Input(shape=input_dims))
    for size in layerSizes:
        model.add(keras.layers.Dense(size))
        model.add(keras.layers.LeakyReLU(alpha=0.05))

    model.add(keras.layers.Dense(output_dims,activation=activation))

    model.compile(optimizer=Adam(learning_rate=lr), loss=loss)

    return model

In [32]:
california_housing.data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [33]:
california_housing.target.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

In [34]:
scaler = StandardScaler()
X = california_housing.data
y = california_housing.target
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[ 2.34476576,  0.98214266,  0.62855945, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 2.33223796, -0.60701891,  0.32704136, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.7826994 ,  1.85618152,  1.15562047, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [-1.14259331, -0.92485123, -0.09031802, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [-1.05458292, -0.84539315, -0.04021111, ..., -0.09122515,
         1.77823747, -0.87362627],
       [-0.78012947, -1.00430931, -0.07044252, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=42)
X_train.shape[1]

8

In [36]:
(X_train.shape[1],)

(8,)

In [43]:
model = build_sequential(lr=0.0001, input_dims=(X_train.shape[1],),output_dims=1, layerSizes=(128,64,32,16,8,4,2),activation=None,loss="MSE")

In [44]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train,y_train, epochs=500, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/500
549/549 [==============================] - 2s 3ms/step - loss: 4.1014 - val_loss: 1.0759
Epoch 2/500
549/549 [==============================] - 1s 2ms/step - loss: 1.0202 - val_loss: 0.7542
Epoch 3/500
549/549 [==============================] - 1s 3ms/step - loss: 0.7993 - val_loss: 0.6763
Epoch 4/500
549/549 [==============================] - 1s 2ms/step - loss: 0.6997 - val_loss: 0.6245
Epoch 5/500
549/549 [==============================] - 1s 2ms/step - loss: 0.6311 - val_loss: 0.5857
Epoch 6/500
549/549 [==============================] - 1s 2ms/step - loss: 0.5823 - val_loss: 0.5560
Epoch 7/500
549/549 [==============================] - 1s 2ms/step - loss: 0.5460 - val_loss: 0.5332
Epoch 8/500
549/549 [==============================] - 2s 3ms/step - loss: 0.5180 - val_loss: 0.5121
Epoch 9/500
549/549 [==============================] - 1s 2ms/step - loss: 0.4950 - val_loss: 0.4969
Epoch 10/500
549/549 [==============================] - 1s 2ms/step - loss: 0.4784 - val_lo

In [ ]:
def build_cnn(output_dims, input_shape=(32,32,3), activation="softmax", loss = tf.keras.losses.SparseCategoricalCrossentropy()):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(output_dims, activation=activation))

    model.compile(optimizer='adam',
                  loss=loss,
                  metrics=['accuracy'])
    return model

In [27]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [32]:
model = build_cnn(10)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1024)             

In [34]:
model = build_cnn(10)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels), callbacks=[callback])

Epoch 1/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.7858 - accuracy: 0.3773 - val_loss: 1.4198 - val_accuracy: 0.4772
Epoch 2/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.3482 - accuracy: 0.5181 - val_loss: 1.3441 - val_accuracy: 0.5355
Epoch 3/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.1870 - accuracy: 0.5824 - val_loss: 1.1655 - val_accuracy: 0.5967
Epoch 4/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.0817 - accuracy: 0.6223 - val_loss: 1.1306 - val_accuracy: 0.6081
Epoch 5/10
1563/1563 [==============================] - 35s 23ms/step - loss: 0.9976 - accuracy: 0.6524 - val_loss: 1.1161 - val_accuracy: 0.6223
Epoch 6/10
1563/1563 [==============================] - 36s 23ms/step - loss: 0.9307 - accuracy: 0.6767 - val_loss: 1.0942 - val_accuracy: 0.6200
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: 0.8707 - accuracy: 0.6954 - val_loss: 1.0823 -

In [2]:
0.27**0.5

0.5196152422706632